# Table of Contents:
* [0 -Importing Libraries](#0)
* [1 - Read in df](#1)
* [2 - PACEU](#2)
* [3 - Sensitivy Analysis](#3)

# 0) Imports <a class="anchor" id="0"></a>

In [1]:
import pandas as pd
import numpy as np
from numpy import nan
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import ast  
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit
from tqdm import tqdm
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

# 1) Read in df <a class="anchor" id="1"></a>

In [2]:
txt_file = 'small.txt'

In [3]:
# Initialize an empty list to store the df
df = []

# Define keys based on the train_train_train_train_train_dfset format
keys = ['#*', '#@', '#t', '#c', '#index', '#%', '#!']

# Open the file and read the contents
with open(txt_file, 'r', encoding='ISO-8859-1') as file:
    current_paper = {}
    for line in file:
        line = line.strip()
        if line.startswith('#*'):
            # New paper starts; save the previous one if it exists
            if current_paper:
                df.append(current_paper)
                current_paper = {}
            current_paper['Title'] = line[2:].strip()
        elif line.startswith('#@'):
            current_paper['Authors'] = line[2:].strip()
        elif line.startswith('#t'):
            current_paper['Year'] = line[2:].strip()
        elif line.startswith('#c'):
            current_paper['Venue'] = line[2:].strip()
        elif line.startswith('#index'):
            current_paper['Index ID'] = line[6:].strip()
        elif line.startswith('#%'):
            if 'References' not in current_paper:
                current_paper['References'] = []
            current_paper['References'].append(line[2:].strip())
        elif line.startswith('#!'):
            current_paper['Abstract'] = line[2:].strip()
        elif line.isdigit() and current_paper:  # Handle end of current paper
            df.append(current_paper)
            current_paper = {}
    # Add the last paper
    if current_paper:
        df.append(current_paper)

# Convert list of dicts to dfFrame
df = pd.DataFrame(df)

# Replacing missing keys with None which becomes NaN in dfFrame
for key in ['Title', 'Authors', 'Year', 'Venue', 'Index ID', 'References', 'Abstract']:
    if key not in df.columns:
        df[key] = None

In [4]:
# # Take a random sample of 10% of the dataset
# df_old = df.copy()

In [5]:
# df = df_old.sample(frac=0.01).reset_index(drop=True)

In [6]:
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
df = df[(df['Year']>1990) & (df['Year']<2010)]
df['Index ID'] = pd.to_numeric(df['Index ID'], errors='coerce')
# Apply the function to the dfFrame column
df['References'] = df['References'].apply(
    lambda x: [int(i) for i in x] if isinstance(x, list) else x
)

Last Cited Year (for SIR)

In [7]:
# Creating a dictionary from the dfFrame assuming 'Index ID' and 'Year' are columns in your dfFrame
year_dict = df.set_index('Index ID')['Year'].to_dict()

def get_last_cited_year(ref_ids):
    if not isinstance(ref_ids, list):
        return np.nan
    # Filter and collect years where reference IDs exist in year_dict
    years = [year_dict.get(int(ref_id)) for ref_id in ref_ids if int(ref_id) in year_dict]
    return max(years) if years else np.nan

# Apply the function to the 'References' column to compute the 'Last Cited Year'
df['Last Cited Year'] = df['References'].apply(get_last_cited_year)
df["Venue"] = df["Venue"].astype(str)
df['Venue'] = df['Venue'].replace('', np.nan)
df["References Count"] = df["References"].apply(lambda x: len(x) if isinstance(x, list) else 0)

In [8]:
# Calculate total authors
df['Authors'] = df['Authors'].fillna('')  # Fill empty strings where there are no authors
unique_authors = set()
df['Authors'].apply(lambda x: unique_authors.update(x.split(',')) if x else None)
total_authors = len(unique_authors) - 1 if '' in unique_authors else len(unique_authors)

# Fill missing Venue values with "Unknown"
df['Venue'] = df['Venue'].fillna("Unknown")

In [9]:
df['Index ID'] = pd.to_numeric(df['Index ID'], errors='coerce')

# Convert 'References' from string representation of list to actual list
def safe_eval(x):
    try:
        return ast.literal_eval(x) if isinstance(x, str) else x
    except:
        return []  # Return an empty list if there's any error

df['References'] = df['References'].apply(safe_eval)

# Creating a dictionary of publication years for quick lookup
publication_years = df.set_index('Index ID')['Year'].to_dict()

# Function to count citations within three years
def count_citations_within_3_years(row):
    if isinstance(row['References'], list):  # Ensure that the data is list
        citation_years = [publication_years.get(int(ref)) for ref in row['References'] if publication_years.get(int(ref)) is not None]
        return sum((year is not None and (row['Year'] <= year <= row['Year'] + 3)) for year in citation_years)
    return 0  # Return 0 if References is not a list

# Apply the function
df['citation_count_within_3_years'] = df.apply(count_citations_within_3_years, axis=1)

In [10]:
expanded_df = pd.DataFrame([
    {**row, 'Simulation_Year': year, 'State': 'Published' if year == row['Year'] else None}
    for _, row in df.iterrows()
    for year in range(row['Year'], 2010)
])

# Define the high-impact check as a function for clarity
def is_high_impact(venue):
    high_impact_venues = [
        "Nature", "Science", "IEEE Transactions on Information Forensics and Security",
        "Journal of Machine Learning Research", "Communications of the ACM",
        "IEEE Transactions on Knowledge and Data Engineering", "Journal of the ACM",
        "Proceedings of the IEEE", "ACM Transactions on Algorithms",
        "IEEE Trans. Parallel Distrib. Syst.", "IEEE Trans. Evolutionary Computation",
        "Journal of Computational Physics", "Journal of the American Society for Information Science and Technology"
    ]
    return venue in high_impact_venues

# Determine initial states based on whether the paper is high impact and citations within 3 years
def determine_initial_state(row):
    if row['Simulation_Year'] == row['Year']:  # Only in the publication year
        if is_high_impact(row['Venue']) and row['citation_count_within_3_years'] >= 5:
            return 'Exposed'
        return 'Published'
    return None  # For all other years, initialize as None which will be updated later

expanded_df['State'] = expanded_df.apply(determine_initial_state, axis=1)

# You might want to fill forward the states based on the previous state
# to ensure continuity until explicitly changed by state update logic
expanded_df.sort_values(by=['Index ID', 'Simulation_Year'], inplace=True)
expanded_df['State'] = expanded_df.groupby('Index ID')['State'].ffill()


In [12]:
len(expanded_df)

488903

In [13]:
# Initialize states only for the publication year
def determine_initial_state(row):
    if row['High_Impact'] and row['citation_count_within_3_years'] >= 5:
        return 'Exposed'
    return 'Published'

# Apply initial state only in the publication year
expanded_df['State'] = expanded_df.apply(lambda row: determine_initial_state(row) if row['Simulation_Year'] == row['Year'] else np.nan, axis=1)

# Ensure DataFrame is sorted by 'Index ID' and 'Simulation_Year' for correct state propagation
expanded_df.sort_values(by=['Index ID', 'Simulation_Year'], inplace=True)

# Forward fill states to carry previous valid state forward until a new state is defined
expanded_df['State'] = expanded_df.groupby('Index ID')['State'].ffill()

# Define the vectorized update function to apply state transitions
def apply_state_transitions(df):
    # Update states based on previous state
    conditions = [
        (df['State'] == 'Published') & (df['citation_count_within_3_years'] >= 5),
        (df['State'] == 'Published') & (df['References'].apply(lambda x: isinstance(x, list))),
        (df['State'] == 'Exposed') & ((df['Simulation_Year'] - df['Year']) > 3) & (df['citation_count_within_3_years'] < 5)
    ]
    choices = ['Exposed', 'Cited', 'Archived']
    df['State'] = np.select(conditions, choices, default=df['State'])
    return df

# Apply state transitions across the dataframe
expanded_df = apply_state_transitions(expanded_df)

In [14]:
# Ensure the dataframe is correctly sorted
expanded_df.sort_values(by=['Index ID', 'Simulation_Year'], inplace=True)

# Precompute whether references are a list for use in conditions
expanded_df['References_Is_List'] = expanded_df['References'].apply(lambda x: isinstance(x, list))

# Create a new column for the previous state only once
expanded_df['Previous_State'] = expanded_df.groupby('Index ID')['State'].shift(1)

# Define conditions using vectorized operations for better performance
def vectorized_update_state(df):
    conditions = [
        (df['Previous_State'] == 'Published') & (df['citation_count_within_3_years'] >= 5) & df['References_Is_List'],
        (df['Previous_State'] == 'Exposed') & ((df['Simulation_Year'] - df['Year']) > 3) & (df['citation_count_within_3_years'] < 5)
    ]
    choices = ['Exposed', 'Archived']
    df['State'] = np.select(conditions, choices, default=df['Previous_State'])
    return df

# Filter for active papers only once
active_papers_df = expanded_df[expanded_df['State'].isin(['Published', 'Exposed', 'Cited'])]

# Convert the active DataFrame to a Dask DataFrame for parallel processing
dask_df = dd.from_pandas(active_papers_df, npartitions=10)

# Set up the progress bar to monitor the compute operation
with ProgressBar():
    # Apply the vectorized update function in parallel and compute the results
    final_active_df = dask_df.map_partitions(vectorized_update_state).compute()

[########################################] | 100% Completed | 208.60 ms


In [15]:
final_df = pd.concat([final_active_df, expanded_df[~expanded_df.index.isin(final_active_df.index)]])

C:\Users\AndrewsMatthewHaines\AppData\Local\Temp\ipykernel_33568\2867724789.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_active_df, expanded_df[~expanded_df.index.isin(final_active_df.index)]])


In [16]:
final_df.to_csv('final_df.csv', index=False)

# 2) PACEU model <a class="anchor" id="2"></a>

In [ ]:
df_ = df.copy()

In [17]:
# Reset index
df = final_df.sample(frac=1).reset_index(drop=True)

In [18]:
min_year = 2000
test_year = 2007
max_year = 2009

In [19]:
# Time-based split
train_df = df[(df['Year'] <= test_year) & (df['Year'] > min_year)]
test_df = df[df['Year'] > test_year]

## 2.1) Define model and calculate transition rates

In [20]:
def citation_model(y, t, lambda_PE, lambda_EC, lambda_CH, gamma_HA):
    P, E, C, H, A = y
    dP_dt = -lambda_PE * P  # Rate at which papers move from PENDING to EXPOSED
    dE_dt = lambda_PE * P - lambda_EC * E  # New papers becoming EXPOSED and some moving to CITED
    dC_dt = lambda_EC * E - lambda_CH * C  # EXPOSED papers moving to CITED or to HIGH IMPACT
    dH_dt = lambda_CH * C - gamma_HA * H  # CITED papers becoming HIGH IMPACT and some moving to ARCHIVED
    dA_dt = gamma_HA * H  # HIGH IMPACT papers becoming ARCHIVED
    return [dP_dt, dE_dt, dC_dt, dH_dt, dA_dt]

In [21]:
def calculate_transition_rates(data, from_state, to_state):
    # Adjust data to match state names and logic
    data['Next_Year'] = data['Year'] + 1
    from_next_year = data[['Index ID', 'Next_Year', to_state]].rename(columns={'Next_Year': 'Year', to_state: 'To_State_Next_Year'})
    merged = data.merge(from_next_year, on=['Index ID', 'Year'], how='left')
    
    # Calculate transitions based on the new state definitions
    transitions = merged[(merged['State'] == from_state) & (merged['To_State_Next_Year'] == to_state)]
    total_from_state = merged[merged['State'] == from_state]
    
    # Avoid division by zero
    rate = len(transitions) / len(total_from_state) if len(total_from_state) > 0 else 0
    return rate

# Calculate parameter estimates for each transition
lambda_PE = calculate_transition_rates(train_df, 'Published', 'Exposed')
lambda_EC = calculate_transition_rates(train_df, 'Exposed', 'Cited')
lambda_CH = calculate_transition_rates(train_df, 'Cited', 'High Impact')
gamma_HA = calculate_transition_rates(train_df, 'High Impact', 'Archived')

# Print the estimated parameters
print(f"Estimated lambda_PE: {lambda_PE}")
print(f"Estimated lambda_EC: {lambda_EC}")
print(f"Estimated lambda_CH: {lambda_CH}")
print(f"Estimated gamma_HA: {gamma_HA}")

C:\Users\AndrewsMatthewHaines\AppData\Local\Temp\ipykernel_33568\3263877288.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Next_Year'] = data['Year'] + 1


KeyError: "['Exposed'] not in index"

### For R0

In [ ]:
def calculate_transition_probability(data, from_state, to_state):
    # Filter data for relevant transitions
    relevant_data = data[data['State'] == from_state]
    transitions = relevant_data[relevant_data['Next_State'] == to_state]

    # Calculate transition probability
    probability = len(transitions) / len(relevant_data) if len(relevant_data) > 0 else 0
    return probability

# Calculate the probability of moving from Exposed to High Impact directly
probability_EH = calculate_transition_probability(expanded_df, 'Exposed', 'High Impact')
print(f"Probability of moving from Exposed to High Impact: {probability_EH:.2f}")

## 2.2) Set initial conditions and run on the training set

In [ ]:
# Initial conditions based on the first year data in training set
initial_conditions_train = [
    train_df[train_df['Year'] == min_year]['State'].value_counts().get('Published', 0),
    train_df[train_df['Year'] == min_year]['State'].value_counts().get('Exposed', 0),
    train_df[train_df['Year'] == min_year]['State'].value_counts().get('Cited', 0),
    train_df[train_df['Year'] == min_year]['State'].value_counts().get('High Impact', 0)
]

# Time points
time_points_train = np.arange(min_year, test_year + 1)

# Solve ODE for training data
results_train = odeint(citation_model, initial_conditions_train, time_points_train, args=(lambda_PE, lambda_EC, lambda_CH, gamma_HA))

# Gather actual data for training period
actual_data_train = [
    [train_df[train_df['Year'] == year]['State'].value_counts().get('Published', 0),
     train_df[train_df['Year'] == year]['State'].value_counts().get('Exposed', 0),
     train_df[train_df['Year'] == year]['State'].value_counts().get('Cited', 0),
     train_df[train_df['Year'] == year]['State'].value_counts().get('High Impact', 0)] for year in time_points_train
]

# Calculate MSE for training
mse_train = np.mean((np.array(actual_data_train) - results_train)**2)

# Plot training results
plt.figure(figsize=(14, 7))
plt.plot(time_points_train, results_train, marker='o', label=['Simulated Published', 'Simulated Exposed', 'Simulated Cited', 'Simulated High Impact'])
plt.plot(time_points_train, np.array(actual_data_train), linestyle='--', label=['Actual Published', 'Actual Exposed', 'Actual Cited', 'Actual High Impact'])
plt.title('Training Set: Actual vs. Simulated Data')
plt.xlabel('Year')
plt.ylabel('Number of Papers')
plt.legend()
plt.grid(True)
plt.show()

print(f"Mean Squared Error for Training Data: {mse_train}")

## 2.3) Test set

In [ ]:
# Initial conditions based on last year of training data
initial_conditions_test = results_train[-1, :]

# Time points for testing data
time_points_test = np.arange(test_year + 1, max_year + 1)

# Solve ODE for testing data
results_test = odeint(citation_model, initial_conditions_test, time_points_test, args=(lambda_PE, lambda_EC, lambda_CH, gamma_HA))

# Gather actual data for testing period
actual_data_test = [
    [test_df[test_df['Year'] == year]['State'].value_counts().get('Published', 0),
     test_df[test_df['Year'] == year]['State'].value_counts().get('Exposed', 0),
     test_df[test_df['Year'] == year]['State'].value_counts().get('Cited', 0),
     test_df[test_df['Year'] == year]['State'].value_counts().get('High Impact', 0)] for year in time_points_test
]

# Calculate MSE for testing
mse_test = np.mean((np.array(actual_data_test) - results_test)**2)

# Plot testing results
plt.figure(figsize=(14, 7))
plt.plot(time_points_test, results_test, marker='o', label=['Simulated Published', 'Simulated Exposed', 'Simulated Cited', 'Simulated High Impact'])
plt.plot(time_points_test, np.array(actual_data_test), linestyle='--', label=['Actual Published', 'Actual Exposed', 'Actual Cited', 'Actual High Impact'])
plt.title('Testing Set: Actual vs. Simulated Data')
plt.xlabel('Year')
plt.ylabel('Number of Papers')
plt.legend()
plt.grid(True)
plt.show()

print(f"Mean Squared Error for Testing Data: {mse_test}")

# 3) Sensitivity <a class="anchor" id="3"></a>

In [ ]:
# Define a range for each parameter
lambda_PE_values = np.linspace(0.05, 0.15, 5)
lambda_EC_values = np.linspace(0.02, 0.08, 5)
lambda_CH_values = np.linspace(0.01, 0.05, 5)
gamma_HA_values = np.linspace(0.005, 0.015, 5)

# Store results
results = []

for lambda_PE in lambda_PE_values:
    for lambda_EC in lambda_EC_values:
        for lambda_CH in lambda_CH_values:
            for gamma_HA in gamma_HA_values:
                # Solve ODE with current parameters
                results_sim = odeint(citation_model, initial_conditions_train, time_points_train, args=(lambda_PE, lambda_EC, lambda_CH, gamma_HA))
                mse_sim = np.mean((np.array(actual_data_train) - results_sim)**2)
                results.append((lambda_PE, lambda_EC, lambda_CH, gamma_HA, mse_sim))

results_df = pd.DataFrame(results, columns=['lambda_PE', 'lambda_EC', 'lambda_CH', 'gamma_HA', 'MSE'])

# Find the parameters that minimize MSE
min_mse_params = results_df.loc[results_df['MSE'].idxmin()]

print("Optimal parameters:")
print(min_mse_params)